In [ ]:
from data_collection.core.currency import CurrencyPair
from datetime import date, timedelta
from typing import *

import requests
import re
import pandas as pd

<h4>Collect all tickers available on Binance</h4>

In [ ]:
resp = requests.get("https://api.binance.com/api/v3/exchangeInfo")

data: Dict[str, Any] = resp.json()

In [ ]:
currency_pairs: List[CurrencyPair] = [
    CurrencyPair(base=entry["baseAsset"], term=entry["quoteAsset"]) for entry in data["symbols"]
]

<h4>Reverse engineer BinanceDataVision</h4>

In [ ]:
params: Dict[str, Any] = {
    "delimiter": "/",
    "prefix": "data/spot/daily/trades/1INCHBUSD/"
}

resp = requests.get(
    "https://s3-ap-northeast-1.amazonaws.com/data.binance.vision",
    params=params
)

In [ ]:
hrefs: List[str] = re.findall(pattern=r"<Key>(.*?)</Key>", string=resp.text)
hrefs = [href for href in hrefs if "CHECKSUM" not in href]

len(hrefs)

In [ ]:
hrefs[1]

In [ ]:
from data_collection.core.time_utils import Bounds, get_last_day_month, get_first_day_month
from calendar import monthrange

start_date: date = date(2023, 1, 28)
end_date: date = date(2024, 3, 3)

bounds: Bounds = Bounds(start_date=start_date, end_date=end_date)

get_last_day_month(bounds.start_date) + timedelta(days=1)

In [ ]:
pd.date_range(start_date, end_date, freq="MS", inclusive="both")

In [ ]:
def generate_year_months(start_date: date, end_date: date) -> Optional[List[date]]:
    date_months: List[date] = [
        date.date() for date in pd.date_range(start_date, end_date, freq="MS", inclusive="both").tolist()
    ]
    # check if the last value is correct
    if not date_months:
        return

    if date_months[-1] == get_first_day_month(end_date):
        date_months.pop(-1)
    return date_months

In [ ]:
generate_year_months(start_date=start_date, end_date=end_date)

In [ ]:
[
    el.date() for el in pd.date_range(get_first_day_month(end_date), end_date, freq="D")
]

In [10]:
import os

os.chdir("C:/Users/mihai/PycharmProjects/microstructure_price_prediction/src")

In [28]:
from core.columns import *
from typing import *
import polars as pl


cols: List[str] = [PRICE, QUANTITY, TRADE_TIME, IS_BUYER_MAKER]

df: pl.LazyFrame = pl.scan_parquet(
    "D:/data/parquet_data/BTCUSDT/BTCUSDT-trades-2024-10.parquet"
)
df.with_row_index()